In [ ]:
# -*- coding: utf-8 -*-
import xarray as xr
import numpy as np
import numpy.ma as ma
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.path as mpath
import glob
import calendar
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import xesmf as xe
import scipy.stats
import os.path
import netCDF4
# from netCDF4 import Dataset#, num2date
# import dask
from datetime import datetime
#np.seterr(divide='ignore', invalid='ignore')
# %matplotlib inline

In [ ]:
# Path to all models
path_ecearth = '/esarchive/exp/CMIP6/dcppA-hindcast/ec-earth3/cmip6-dcppA-hindcast_i1p1/DCPP/EC-Earth-Consortium/EC-Earth3/dcppA-hindcast/r2i1p1f1/SImon/siconc/gn/v20200101/'
path_canesm5 = '/esarchive/exp/CMIP6/dcppA-hindcast/canesm5/cmip6-dcppA-hindcast_i1p2/DCPP/CCCma/CanESM5/dcppA-hindcast/r2i1p2f1/SImon/siconc/gn/v20200101/'
path_norcpm1_1 = '/esarchive/exp/CMIP6/dcppA-hindcast/norcpm1/cmip6-dcppA-hindcast_i1p1/DCPP/NCC/NorCPM1/dcppA-hindcast/r2i1p1f1/SImon/siconc/gn/v20200101/'
path_norcpm1_2 = '/esarchive/exp/CMIP6/dcppA-hindcast/norcpm1/cmip6-dcppA-hindcast_i2p1/DCPP/NCC/NorCPM1/dcppA-hindcast/r2i2p1f1/SImon/siconc/gn/v20200101/'
path_ipsl = '/esarchive/exp/CMIP6/dcppA-hindcast/ipsl-cm6a-lr/cmip6-dcppA-hindcast_i1p1/DCPP/IPSL/IPSL-CM6A-LR/dcppA-hindcast/r2i1p1f1/SImon/siconc/gn/v20200101/'
path_miroc = '/esarchive/exp/CMIP6/dcppA-hindcast/miroc6/cmip6-dcppA-hindcast_i1p1/DCPP/MIROC/MIROC6/dcppA-hindcast/r2i1p1f1/SImon/siconc/gn/v20200101/'
path_mpi = '/esarchive/exp/CMIP6/dcppA-hindcast/mpi-esm1-2-hr/cmip6-dcppA-hindcast_i1p1/DCPP/MPI-M/MPI-ESM1-2-HR/dcppA-hindcast/r2i1p1f1/SImon/siconc/gn/v20200101/'

In [ ]:
# SET OPTIONS #

# Dictionary with model names and consortiums for paths
model_dic = {'ecearth': ['ec-earth3', 'i2p1', 'EC-Earth-Consortium/EC-Earth3', 'EC-Earth3'], 'norcpm1_1': ['norcpm1', 'i1p1', 'NCC/NorCPM1'],
             'norcpm1_2': ['norcpm1', 'i2p1', 'NCC/NorCPM1'], 'canesm': ['canesm5', 'i1p2', 'CCCma/CanESM5'],
             'ipsl_cm': ['ipsl-cm6a-lr', 'i1p1', 'IPSL/IPSL-CM6A-LR'], 'miroc': ['miroc6', 'i1p1', 'MIROC/MIROC6'],
             'mpi': ['mpi-esm1-2-hr', 'i1p1', 'MPI-M/MPI-ESM1-2-HR']}

path1 = '/esarchive/exp/CMIP6/dcppA-hindcast/'
path2 = 'cmip6-dcppA-hindcast_' 
path3 = 'dcppA-hindcast'
compo = 'SImon'
mod_compo = 'seaIce'
vdate = 'v20200101/'
var = 'siconc'
pathout = '/esarchive/scratch/ruben/dcpp/data/'
pathout_plot = '/esarchive/scratch/ruben/dcpp/plots/'

# Members EC-Earth
mem_pred1_ecearth = 6
mem_pred2_ecearth = 10
members_pred_ecearth = []
for r in range(mem_pred1_ecearth, mem_pred2_ecearth+1):
    me_pred_ecearth = 'r{0}i2p1f1'.format(r)
    members_pred_ecearth.append(me_pred_ecearth)

year1 = 1970 #61
year2 = 2010 #00
# Chunks EC-Earth
chunks_pred_ecearth = []
for j in range(year1, year2+1):
    chupred = '{0}01-{1}12'.format(str(j), str(j))
    chunks_pred_ecearth.append(chupred)
    
print(chunks_pred_ecearth)

# Chunks HadiSST
chunks_hadisst = []
for j in range(year1, year2+1):
    for t in range(1, 13):
        chu_disst = '{0}{1}'.format(str(j), str(t).zfill(2))
        chunks_hadisst.append(chu_disst)

#print(chunks_hadisst)  

In [ ]:
# Load HadISST # (Continous Timeseries)
hadisst_path = '/esarchive/obs/ukmo/hadisst_v2.2/monthly_mean/sic/'

filenames_hadisst = []
for chu in chunks_hadisst:
    filename_ = glob.glob(hadisst_path + 'sic_' + chu + '.nc')
    filenames_hadisst.append(filename_[0])
    
list_hadisst = []
for file_ in filenames_hadisst:
    date_hadisst = file_.split('sic_')[1].split('.')[0]
    
    ds = xr.open_dataset(file_, decode_times=False)
    mon_index = xr.cftime_range(date_hadisst, periods=1, freq='MS')
    ds.coords['time'] = mon_index
    list_hadisst.append(ds)
    
ds_hadisst = xr.concat(list_hadisst, dim='time')
ds_hadisst = ds_hadisst.sortby('time') # 124 Mb

In [ ]:
# Histo2Hindcast #
ds_hadisst2 = ds_hadisst['sic']*100 #.to_array()

formatted_hadisst = []
for s_dates in range(0, len(chunks_pred_ecearth)):
    forecast_period = chunks_pred_ecearth[s_dates]
    first_step = forecast_period.split('-')[0]
    last_step = forecast_period.split('-')[1]
    start_slice = '-'.join([first_step[:4], first_step[-2:]])
    end_slice = '-'.join([last_step[:4], last_step[-2:]])
    sele = ds_hadisst2.sel(time=slice(start_slice, end_slice))
    sele['time'] = np.arange(1, 13) # ftime
    formatted_hadisst.append(sele)
    
sdates_dim_hadisst = xr.DataArray(chunks_pred_ecearth, name='sdate', dims='sdate')
reshaped_hadisst = xr.concat(formatted_hadisst, dim=sdates_dim_hadisst)

## Load EC-Earth (the only with chunks of 1 year, all the rest 10 years)

In [ ]:
# Load EC-Earth # 1st year

path_saving_ecearth = '{0}{1}_pred_ecearth_mean_{2}-{3}_ensemble_year1.nc'.format(pathout, var, str(year1), str(year2))

if os.path.isfile(path_saving_ecearth):
    print('File exists. Loading... ') # 40x12x292x362
    ecearth_ensemble = xr.open_dataarray(path_saving_ecearth, decode_times=False)
else:
    print('IT DOES NOT EXIST') # Load everything
    HIND_ecearth = []
    for mem in members_pred_ecearth:
        filenames_pred = []
        for chu in chunks_pred_ecearth:

            start_date = int(chu.split('01-')[0]) - 1

            file_path_pred = '{0}{1}/{2}{3}{4}{5}/{6}/{7}/{8}/{9}/gn/{10}'.format(path1, model_dic['ecearth'][0], path2,                     
                                                                                model_dic['ecearth'][1], '/DCPP/',
                                                                                model_dic['ecearth'][2], path3, mem, compo, 
                                                                                var, vdate)

            filename_ = glob.glob(file_path_pred + var + '_' + compo + '_' + model_dic['ecearth'][3] + '_' + path3 + '_s' +
                                  str(start_date) + '-' + mem + '_gn_' + chu + '.nc')        

            filenames_pred.append(filename_[0])

        f_list_pred = sorted(filenames_pred)

        list_pred = []
        for file_ in f_list_pred:
            ds = xr.open_dataset(file_, decode_times=False, chunks={'time': 1})
            ds['time'] = np.arange(1, 13) # forecast time 1st year
            list_pred.append(ds)

        da_num_pred = xr.DataArray(chunks_pred_ecearth, name='sdate', dims='sdate')
        ds_pred_num = xr.concat(list_pred, dim=da_num_pred)

        HIND_ecearth.append(ds_pred_num)

    hindcast = xr.DataArray(members_pred_ecearth, name='member', dims='member')
    ds_hind = xr.concat(HIND_ecearth, dim=hindcast)
    # Ensemble Mean and Start Date Mean
    ecearth_ensemble = ds_hind[var].mean(dim='member')
    del ds_hind

    # print('Size of the file: ', pred_clim.nbytes/1e6, 'Mb')
    
    ecearth_ensemble.to_netcdf(path = path_saving_ecearth, format = 'NETCDF4')

## Load Rest of Models (chunks of 10 years)

In [ ]:
model_dic_10y = {'canesm': ['canesm5', 'i1p2', 'CCCma/CanESM5', 2, 2, 'CanESM5', 121], # 1 (member1), 1 (member2)
                 'norcpm1_1': ['norcpm1', 'i1p1', 'NCC/NorCPM1', 1, 1, 'NorCPM1', 124],
                 'norcpm1_2': ['norcpm1', 'i2p1', 'NCC/NorCPM1', 1, 1, 'NorCPM1', 124],
                 'ipsl_cm': ['ipsl-cm6a-lr', 'i1p1', 'IPSL/IPSL-CM6A-LR', 1, 1, 'IPSL-CM6A-LR', 121],
                 'miroc': ['miroc6', 'i1p1', 'MIROC/MIROC6', 1, 1, 'MIROC6', 123]
                }

             #'mpi': ['mpi-esm1-2-hr', 'i1p1', 'MPI-M/MPI-ESM1-2-HR']}

mod_to_load = ['miroc']
                
year1=1970
year2=2010

for mod in mod_to_load:

    # Members
    members_pred_mod = []
    for r in range(model_dic_10y[mod][3],model_dic_10y[mod][4]+1):
        me_pred_mod = 'r{0}{1}f1'.format(r, model_dic_10y[mod][1])
        members_pred_mod.append(me_pred_mod)

    # Chunks 
    chunks_pred_mod = []
    for j in range(year1, year2+1):
        if mod == 'canesm' or mod == 'ipsl_cm':
            chupred = '{0}01-{1}12'.format(str(j), str(j+9))
            chunks_pred_mod.append(chupred)
        elif mod == 'norcpm1_1' or mod == 'norcpm1_2':
            chupred = '{0}10-{1}12'.format(str(j-1), str(j+9))
            chunks_pred_mod.append(chupred)
        elif mod == 'miroc':
            chupred = '{0}11-{1}12'.format(str(j-1), str(j+9))
            chunks_pred_mod.append(chupred)
            
#   HIND_mod = []
    for mem in members_pred_mod:

        path_saving_mod10 = '{0}{1}_pred_{2}_{3}_{4}-{5}_10y.nc'.format(pathout, var, mod, mem, str(year1), str(year2))

        if os.path.isfile(path_saving_mod10):
            print('File exists. Loading... ')
            mod = xr.open_dataset(path_saving_mod10, decode_times=False)[var]
        else:
            print('IT DOES NOT EXIST') # Load everything

            HIND_mod = []
            filenames_pred = []
            for chu in chunks_pred_mod:
                if mod == 'canesm' or mod == 'ipsl_cm':
                    start_date = int(chu.split('01-')[0]) - 1
                elif mod == 'norcpm1_1' or mod == 'norcpm1_2':
                    start_date = int(chu.split('10-')[0])
                elif mod == 'miroc':
                    start_date = int(chu.split('11-')[0])
                    

                file_path_pred = '{0}{1}/{2}{3}{4}{5}/{6}/{7}/{8}/{9}/gn/{10}'.format(path1, model_dic_10y[mod][0], path2, model_dic_10y[mod][1], '/DCPP/', model_dic_10y[mod][2], path3, mem, compo, var, vdate)

                filename_ = glob.glob(file_path_pred + var + '_' + compo + '_' + str(model_dic_10y[mod][5]) + '_' + path3 + '_s' + str(start_date) + '-' + mem + '_gn_' + chu + '.nc')

                filenames_pred.append(filename_[0])

            f_list_pred = sorted(filenames_pred)
            list_pred = []
            for file_ in f_list_pred:
                ds = xr.open_dataset(file_, decode_times=False, chunks={'time': 1})
                ds['time'] = np.arange(1, model_dic_10y[mod][6]) # forecast time 1st year
                list_pred.append(ds)

            da_num_pred = xr.DataArray(chunks_pred_mod, name='sdate', dims='sdate')
            ds_pred_num = xr.concat(list_pred, dim=da_num_pred)

            HIND_mod.append(ds_pred_num)

            ds_mod = xr.concat(HIND_mod, dim='time')
            ds_mod.to_netcdf(path_saving_mod10, format = 'NETCDF4')


## Select 1st year from 10

In [ ]:
# For the 10y models (CanESM)
if mod == 'canesm' or mod == 'ipsl_cm':
    sele_mod = mod[:,0:12,:,:]
elif mod == 'norcpm1_1' or mod == 'norcpm1_2':
    sele_mod = mod[:,3:15,:,:] # Los chunks empiezan en octubre
elif mod == 'miroc':
    sele_mod = mod[:,2:15,:,:]

# LACKING MPI
    

# Calculate Anomalies

In [ ]:
# Anomaly Calculation #
#######################
# ~ HadiSST
sd_mean_hadisst = reshaped_hadisst.groupby('time').mean(dim='sdate') # , skipna=True, keep_attrs = True)
ano_hadisst = reshaped_hadisst.groupby('time') - sd_mean_hadisst

# ~ EC-Earth
sd_mean_ecearth = ecearth_ensemble.groupby('time').mean(dim='sdate', keep_attrs = True)
ano_ecearth = ecearth_ensemble.groupby('time') - sd_mean_ecearth

# ~ CanESM
sd_mean_mod = sele_mod.groupby('time').mean(dim='sdate', keep_attrs = True)
ano_mod = sele_mod.groupby('time') - sd_mean_mod


## Interpolate Model Anomalies

In [ ]:
# Models to interpolate #
dic_interp = {'canesm': ano_mod} #'ecearth': ano_ecearth, 'norcpm1_1': 'NorCPM1_1'}
mod_to_interp = ['canesm'] #['ecearth']

In [ ]:
# Interpolate Model Anomalies to regular # ~~~~~~
nr_dates = year2 - year1 + 1

dict_out = {}

for mod in mod_to_interp:

    latitude = dic_interp[mod].latitude
    longitude = dic_interp[mod].longitude

    res_new = 1 # degrees; the resolution of the new grid
    lat_new = -90 + np.arange((2*90)/res_new) 

    lat_new = lat_new*-1  # reverse latitudes 

    lon_new = np.arange((360)/res_new) 
    lon_new, lat_new = np.meshgrid(lon_new,lat_new)

    data_new = np.empty(shape=(nr_dates,12,180,360))

    
    for sdate in range(0, nr_dates):
        for mon in range(0, 12):

            data = np.squeeze(np.array(ano_mod[sdate,mon,:,:])) # 
            ndim0 = np.size(data,axis=0)
            ndim1 = np.size(data,axis=1)

            # Regrid
          #  data_new = data
            lat_temp = latitude.copy()
            lon_temp = longitude.copy()
            method = np.str('nearest_s2d')

            # Make sure the dimensions of the new grid are in the right order
            lat_temp = lat_new.copy()
            lon_temp = lon_new.copy()
            if ndim0>ndim1:
                lat_temp = np.transpose(lat_temp)
                lon_temp = np.transpose(lon_temp)

            lat_temp2 = latitude.copy()
            lon_temp2 = longitude.copy()

            # Create the xarray datasets to feed into the xesmf regridder
            ds_in = xr.Dataset({'lat': (['x','y'],lat_temp2),
                                'lon': (['x','y'],lon_temp2)})
            ds_out = xr.Dataset({'lat': (['x','y'],lat_temp),
                                    'lon': (['x','y'],lon_temp)})
            # Regrid the data
            f = xe.Regridder(ds_in, ds_out, method, periodic=True, reuse_weights=True)
            data_new[sdate,mon,:,:] = f(data)
            
    dict_out['{0}'.format(mod)] = data_new

## Calculate ACC

In [ ]:
# Calculate ACC (Models & Reference) #

ano_had = np.nan_to_num(ano_hadisst)
#np.isnan(ano_had).any()

mod_to_corr = ['canesm'] #['ecearth']
dict_corr = {}
for mod in mod_to_corr:
    
    ano_mod = np.nan_to_num(dict_out[mod])
    
    corr_coef = np.empty(shape=(12, 180, 360))
    pvalue = np.empty(shape=(12, 180, 360))
    for mon in range(0,12):
        for lat in range(0,50): # Only correlations for the Arctic
            for lon in range(0,360):
                corr = scipy.stats.pearsonr(np.squeeze(ano_mod[:,mon,lat,lon]), np.squeeze(ano_had[:,mon,lat,lon])) 
                
                corr_coef[mon,lat,lon] = corr[1]
                pvalue[mon,lat,lon] = corr[0]
    # Only significant values will be kept with 1
    pvalue[pvalue>=0.05] = np.nan
    pvalue[pvalue<0.05] = 1
    dict_corr['{0}'.format(mod)] = (corr_coef, pvalue)

## Save ACC

In [ ]:
mod = 'canesm'
path_saving_acc = '{0}{1}_pred_{2}_{3}_{4}-{5}_ACC_y1.nc'.format(pathout, var, mod, mem, str(year1), str(year2))
path_saving_pval = '{0}{1}_pred_{2}_{3}_{4}-{5}_pvalue_y1.nc'.format(pathout, var, mod, mem, str(year1), str(year2))

array_acc = xr.DataArray(dict_corr[mod][0])
array_acc.to_netcdf(path_saving_acc, format = 'NETCDF4')

array_pval = xr.DataArray(dict_corr[mod][1])
array_pval.to_netcdf(path_saving_pval, format = 'NETCDF4')

## ACC Maps

In [ ]:

month_to_plot = [9] # 3 March / 9 September
for mes in month_to_plot:
    fig1, axes = plt.subplots(figsize=(6,6), constrained_layout=False) # 20,20
    index = 0
    for m in mod_to_plot:
        index += 1
        axes.set_axis_off()

        corr_model = array_acc[mes,:,:]

        # New evenly spaced, monotonically increasing lat-lon grid for regridding the sea ice
        # concentration for the contour plot (otherwise the contour will reverse on itself)
        res_new = 1 # degrees; the resolution of the new grid
        lat_new = -90 + np.arange((2*90)/res_new) # 1+(2*90)

        lon_new = np.arange((360)/res_new) # 1 +(360
        lon_new, lat_new = np.meshgrid(lon_new,lat_new)

        ice_edge_conc = 15 # Sea Ice Edge # I would have to plot with clim SIC !!

        # Compute a circle in axes coordinates, which we can use as a boundary
        # for the map. We can pan/zoom as much as we like - the boundary will be
        # permanently circular.
        theta = np.linspace(0, 2*np.pi, 100)
        center, radius = [0.5, 0.5], 0.5
        verts = np.vstack([np.sin(theta), np.cos(theta)]).T
        circle = mpath.Path(verts * radius + center)
        # Make a pcolor plot of sea ice concentration, and contour the sea ice edge
        ax = fig1.add_subplot(1, 1, index, projection = ccrs.NorthPolarStereo()) # 5, 4
        ax.add_feature(cfeature.LAND,zorder=100,edgecolor='black',facecolor='Grey')
        ax.set_boundary(circle, transform=ax.transAxes)
        ax.set_extent([-180, 180, 55, 90], crs=ccrs.PlateCarree())
        this = ax.pcolormesh(lon_temp,lat_temp, corr_model, transform=ccrs.PlateCarree(),
                             cmap = plt.cm.RdYlBu_r, vmin=-1, vmax=1)
        # this2 = ax.contour(lon_temp,lat_temp,mydata_new,15,transform=ccrs.PlateCarree(),colors='green')
        cf = ax.scatter(lon_temp, lat_temp, s=array_pval[mes,:,], color='#000000', marker='^', transform=ccrs.PlateCarree())
#         plt.title('ACC %s' % (dict_title[m]))
    
    fig1.colorbar(this, ax=axes, orientation='horizontal', fraction = 0.09, label = 'ACC')#, shrink=-0.6)
#     plt.show()
    plt.savefig('{0}ACC_{1}_canesm.eps'.format(pathout_plot, str(mes)))










In [ ]:
# MAP of ACC:
#~~~~~~~~~~~~~#

dict_title = {'ecearth': 'EC-Earth', 'canesm': 'CanESM'  ,'norcpm1_1': 'NorCPM1_1'}
mod_to_plot = ['canesm'] #['ecearth']

month_to_plot = [9] # 3 March / 9 September
for mes in month_to_plot:
    fig1, axes = plt.subplots(figsize=(6,6), constrained_layout=False) # 20,20
    index = 0
    for m in mod_to_plot:
        index += 1
        axes.set_axis_off()

        corr_model = dict_corr[m][0][mes,:,:]

        # New evenly spaced, monotonically increasing lat-lon grid for regridding the sea ice
        # concentration for the contour plot (otherwise the contour will reverse on itself)
        res_new = 1 # degrees; the resolution of the new grid
        lat_new = -90 + np.arange((2*90)/res_new) # 1+(2*90)

        lon_new = np.arange((360)/res_new) # 1 +(360
        lon_new, lat_new = np.meshgrid(lon_new,lat_new)

        ice_edge_conc = 15 # Sea Ice Edge # I would have to plot with clim SIC !!

        # Compute a circle in axes coordinates, which we can use as a boundary
        # for the map. We can pan/zoom as much as we like - the boundary will be
        # permanently circular.
        theta = np.linspace(0, 2*np.pi, 100)
        center, radius = [0.5, 0.5], 0.5
        verts = np.vstack([np.sin(theta), np.cos(theta)]).T
        circle = mpath.Path(verts * radius + center)
        # Make a pcolor plot of sea ice concentration, and contour the sea ice edge
        ax = fig1.add_subplot(1, 1, index, projection = ccrs.NorthPolarStereo()) # 5, 4
        ax.add_feature(cfeature.LAND,zorder=100,edgecolor='black',facecolor='Grey')
        ax.set_boundary(circle, transform=ax.transAxes)
        ax.set_extent([-180, 180, 55, 90], crs=ccrs.PlateCarree())
        this = ax.pcolormesh(lon_temp,lat_temp, corr_model, transform=ccrs.PlateCarree(),
                             cmap = plt.cm.RdYlBu_r, vmin=-1, vmax=1)
        # this2 = ax.contour(lon_temp,lat_temp,mydata_new,15,transform=ccrs.PlateCarree(),colors='green')
        cf = ax.scatter(lon_temp, lat_temp, s=dict_corr[m][1][mes,:,], color='#000000', marker='^', transform=ccrs.PlateCarree())
        plt.title('ACC %s' % (dict_title[m]))
    
    fig1.colorbar(this, ax=axes, orientation='horizontal', fraction = 0.09, label = 'ACC')#, shrink=-0.6)
#     plt.show()
    plt.savefig('{0}ACC_{1}_canesm.eps'.format(pathout_plot, str(mes)))

## Open DataSets (Specific time steps for visualizing SIC)

In [ ]:
# EC-EARTH #
sic_ecearth = xr.open_dataset(path_ecearth + 'siconc_SImon_EC-Earth3_dcppA-hindcast_s1962-r2i1p1f1_gn_196211-196310.nc')
print('Size of the file: ', sic_ecearth.nbytes/1e6, 'Mb')

In [ ]:
# NORCPM1_1 #
sic_norcpm1_1 = xr.open_dataset(path_norcpm1_1 + 'siconc_SImon_NorCPM1_dcppA-hindcast_s1962-r2i1p1f1_gn_196210-197212.nc')

In [ ]:
# NORCPM1_2 #
sic_norcpm1_2 = xr.open_dataset(path_norcpm1_2 + 'siconc_SImon_NorCPM1_dcppA-hindcast_s1962-r2i2p1f1_gn_196210-197212.nc')

In [ ]:
# CanESM5 #
sic_canesm5 = xr.open_dataset(path_canesm5 + 'siconc_SImon_CanESM5_dcppA-hindcast_s1961-r2i1p2f1_gn_196201-197112.nc')

In [ ]:
# IPSL-CM6 #
sic_ipsl = xr.open_dataset(path_ipsl + 'siconc_SImon_IPSL-CM6A-LR_dcppA-hindcast_s1961-r2i1p1f1_gn_196201-197112.nc')

In [ ]:
# MIROC6 # 
sic_miroc = xr.open_dataset(path_miroc + 'siconc_SImon_MIROC6_dcppA-hindcast_s1962-r2i1p1f1_gn_196211-197212.nc')

In [ ]:
# MPI-ESM1 #
sic_mpi = xr.open_dataset(path_mpi + 'siconc_SImon_MPI-ESM1-2-HR_dcppA-hindcast_s1962-r2i1p1f1_gn_196211-197212.nc')

In [ ]:
# NSIDC 0051 monthly data #
dfile = '/esarchive/obs/nasa/nsidc0051_gsfc_nasateam_seaice/processed/north/monthly_mean/siconc_r1i1p1_mon_197901_nh-psn25.nc'
sic_nsidc = xr.open_dataset(dfile)

In [ ]:
# HADISST #
hadisst_path = '/esarchive/obs/ukmo/hadisst_v2.2/monthly_mean/sic/'
sic_hadisst = xr.open_dataset(hadisst_path + 'sic_196101.nc')

In [ ]:
# SELECT DATA TO BE PLOTTED
dic_mod = {'ecearth': sic_ecearth, 'norcpm1_1': sic_norcpm1_1,
             'norcpm1_2': sic_norcpm1_2, 'canesm': sic_canesm5,
             'ipsl_cm': sic_ipsl, 'miroc': sic_miroc,
             'mpi': sic_mpi, 'hadisst': sic_hadisst, 'nsidc': sic_nsidc}
mod_to_plot = ['ecearth', 'norcpm1_1', 'norcpm1_2', 'canesm', 'ipsl-cm', 'miroc', 'mpi', 'hadisst', 'nsidc']

In [ ]:
# ARCTIC # (Plot Models and Observations)
#~~~~~~~~#
# fig1 = plt.figure(figsize=(20,20))
fig1, axes = plt.subplots(figsize=(20,20), constrained_layout=False) #, sharex=True, sharey=True)

i = 1
index = 0
for m in mod_to_plot:
    index += 1
    axes.set_axis_off()

    # Select VAR and
    # Deal with diferent names for latitude and longitude coordinates
    if m == 'hadisst':
        sic_model = dic_mod[m]['sic']*100
        latitude = sic_model.latitude
        longitude = sic_model.longitude
        first_month = sic_model
    else:
        sic_model = dic_mod[m]['siconc']
        if m == 'nsidc':
            first_month = sic_model
            latitude = sic_model.latitude
            longitude = sic_model.longitude
        else:
            first_month = sic_model.sel(time='1962-11-16')
            if np.str(dic_mod[m].source_id) == 'IPSL-CM6A-LR':
                latitude = sic_model.nav_lat
                longitude = sic_model.nav_lon
            else:
                latitude = sic_model.latitude
                longitude = sic_model.longitude
    
    # New evenly spaced, monotonically increasing lat-lon grid for regridding the sea ice
    # concentration for the contour plot (otherwise the contour will reverse on itself)
    res_new = 1 # degrees; the resolution of the new grid
    lat_new = -90 + np.arange((2*90)/res_new) # 1+(2*90)

    lon_new = np.arange((360)/res_new) # 1 +(360
    lon_new, lat_new = np.meshgrid(lon_new,lat_new)

    ice_edge_conc = 15 # the concentration threshold for the sea ice edge

    # Load in the sea ice concentration for the months I want,
    # average it over those months, and find its size
    
    data = np.squeeze(np.array(first_month))
    ndim0 = np.size(data,axis=0)
    ndim1 = np.size(data,axis=1)
    
    if m is not 'hadisst':
        # Regrid
        data_new = data
        lat_temp = latitude.copy()
        lon_temp = longitude.copy()
        method = np.str('bilinear')
        method = np.str('nearest_s2d')

        # Make sure the dimensions of the new grid are in the right order
        lat_temp = lat_new.copy()
        lon_temp = lon_new.copy()
        if ndim0>ndim1:
            lat_temp = np.transpose(lat_temp)
            lon_temp = np.transpose(lon_temp)

        lat_temp2 = latitude.copy()
        lon_temp2 = longitude.copy()
        # lat_temp2[np.abs(lat_temp2)>90] = np.nan
        # lon_temp2[np.abs(lon_temp2)>360] = np.nan #-- > ERROR because it is not a list

        # Create the xarray datasets to feed into the xesmf regridder
        ds_in = xr.Dataset({'lat': (['x','y'],lat_temp2),
                            'lon': (['x','y'],lon_temp2)})
        ds_out = xr.Dataset({'lat': (['x','y'],lat_temp),
                                'lon': (['x','y'],lon_temp)})
        # Regrid the data
        f = xe.Regridder(ds_in, ds_out, method, periodic=True, reuse_weights=True)
        data_new = f(data)

        # A data mask for NortPolarStereo plots
        mydata = ma.masked_where(latitude<0.,data.copy())
        mydata_new = ma.masked_where(lat_temp<0.,data_new.copy())
    elif m == 'hadisst':
        lon_temp = longitude
        lat_temp = latitude
        mydata = data.copy()
        mydata_new = data.copy()
        
    # Compute a circle in axes coordinates, which we can use as a boundary
    # for the map. We can pan/zoom as much as we like - the boundary will be
    # permanently circular.
    theta = np.linspace(0, 2*np.pi, 100)
    center, radius = [0.5, 0.5], 0.5
    verts = np.vstack([np.sin(theta), np.cos(theta)]).T
    circle = mpath.Path(verts * radius + center)
    # Make a pcolor plot of sea ice concentration, and contour the sea ice edge
    ax = fig1.add_subplot(5, 4, index, projection = ccrs.NorthPolarStereo()) # 5, 4
    ax.add_feature(cfeature.LAND,zorder=100,edgecolor='black',facecolor='Grey')
    ax.set_boundary(circle, transform=ax.transAxes)
    ax.set_extent([-180, 180, 55, 90], crs=ccrs.PlateCarree())
    # ax.set_extent([0.005, 360, -90, -50], crs=ccrs.PlateCarree()) # South Pole
    this = ax.pcolormesh(longitude,latitude,mydata,transform=ccrs.PlateCarree(),cmap = plt.cm.Blues_r, vmin=0, vmax=100)
    this2 = ax.contour(lon_temp,lat_temp,mydata_new,[ice_edge_conc],transform=ccrs.PlateCarree(),colors='orange')
    if m == 'nsidc':
        plt.title('NSIDC')
    elif m == 'hadisst':
        plt.title('HadiSST')
    else:
        plt.title(np.str(dic_mod[m].source_id))
    
fig1.colorbar(this, ax=axes, orientation='horizontal', fraction = 0.09, label = 'Sea Ice Concentration (%)')#, shrink=-0.6)
# plt.show()
# plt.savefig('example.png')